In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd

In [17]:
spark = SparkSession.builder.appName("Data Wrangling with Spark SQL").getOrCreate()

user_log = spark.read.json("Data/sparkify_log_small.json")

user_log.createOrReplaceTempView("user_log")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [7]:
spark.sql("""
     SELECT DISTINCT page 
     FROM user_log 
     WHERE page NOT IN (
         SELECT DISTINCT page 
         FROM user_log 
         WHERE userId = ''
         )
         """).show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

We might prefer SQL over data frames since sql is ver commonly used language to query ant database. We may be more more familiar with sql syntax than dataframes.

Spark data frames gives us more control. We can break down queries into smaller steps. We can cache intermediate results or repartition imtermediate results



# Question 3

How many female users do we have in the data set?


In [8]:
spark.sql("""
     SELECT gender, count(DISTINCT userId) as cnt
     FROM user_log
     WHERE gender = 'F'
     GROUP BY gender
     """).show()

+------+---+
|gender|cnt|
+------+---+
|     F|462|
+------+---+



# Question 4 (challenge)


How many songs were played from the most played artist?

In [9]:
spark.sql("""
    SELECT DISTINCT artist, count(ts) as play_cnt
    FROM user_log
    WHERE page = 'NextSong'
    GROUP BY artist
    ORDER BY play_cnt DESC
    LIMIT 1
    """).show()

+--------+--------+
|  artist|play_cnt|
+--------+--------+
|Coldplay|      83|
+--------+--------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [18]:
# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;
is_home = spark.sql("SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home FROM user_log \
            WHERE (page = 'NextSong') or (page = 'Home') \
            ")

# keep the results in a new view
is_home.createOrReplaceTempView("is_home_table")

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql("SELECT *, SUM(is_home) OVER \
    (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
    FROM is_home_table")

# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

# find the average count for NextSong
spark.sql("SELECT AVG(count_results) FROM \
          (SELECT COUNT(*) AS count_results FROM period_table \
GROUP BY userID, period, page HAVING page = 'NextSong') AS counts").show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

